In [16]:
import rankBN
import numpy as np
import qml
import line_profiler
import numba
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [6]:
%lprun -f rankBN.Ranker._get_site_similarity_CM rankBN.do_main("napthalene-molpro.xyz", "napthalene-molpro.mol2", 2.2, "CM") 

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: C.ar
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Working on stoichiometry: [5 5 5 5 5 7 7 7 7 7]
Found 66 molecules from 252 permutations.
55 components found
Group energy -2005.9177820267687
Found: [5, 7, 5, 7, 7, 5, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 5, 7, 5, 7]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 7, 5, 5, 7, 7, 5]
Group energy -1884.1873804971317
Found: [5, 5, 7, 7, 7, 5, 7, 5, 7, 5]
Group energy -1838.4990439770554
Found: [5, 7, 5, 7, 7, 5, 7, 7, 5, 5]
Found: [5, 7, 5, 7, 7, 7, 5, 7, 5, 5]
Group energy -1838.4990439770554
Found: [7, 5, 5, 7, 7, 5, 7, 7, 5, 5]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 7, 5, 7, 7]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 5, 7, 7, 7]
Found: [5, 5, 7, 5, 7, 5, 7, 5, 7, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 5, 5, 7, 7, 7, 5, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 7, 5, 7, 5, 7, 5, 7]
Group energy -1762.4569789674956
Found

Timer unit: 1e-06 s

Total time: 60.7299 s
File: /mnt/c/Users/guido/workcopies/apdft/prototyping/networks/rankBN.py
Function: _get_site_similarity_CM at line 222

Line #      Hits         Time  Per Hit   % Time  Line Contents
   222                                           	def _get_site_similarity_CM(self, nuclear_charges):
   223                                           		""" Returns i, j, distance."""
   224     31116     239035.0      7.7      0.4  		charges = self._c.nuclear_charges.copy().astype(np.float)
   225     31116     133983.0      4.3      0.2  		charges[self._includeonly] = nuclear_charges
   226     31116     247403.0      8.0      0.4  		atoms = np.where(self._c.nuclear_charges == 6)[0]
   227     31116    4829657.0    155.2      8.0  		a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
   228     31116     236370.0      7.6      0.4  		s = np.zeros((self._c.natoms, self._c.natoms))
   229     31116

In [8]:
nuclear_charges, coordinates = rankBN.Ranker.read_xyz("napthalene-molpro.xyz")
r = rankBN.Ranker(nuclear_charges, coordinates, "napthalene-molpro.xyz", "napthalene-molpro.mol2", explain=True, sim=2.2, simmode="CM")

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: C.ar
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [23]:
TESTVECS = get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))

In [206]:
# Original implementation
def get_site_similarity_CM_orig(self, nuclear_charges):
    """ Returns i, j, distance."""
    charges = self._c.nuclear_charges.copy().astype(np.float)
    charges[self._includeonly] = nuclear_charges
    atoms = self._includeonly
    a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    s = np.zeros((self._c.natoms, self._c.natoms))
    s[np.tril_indices(self._c.natoms)] = a
    d = np.diag(s)
    s += s.T
    s[np.diag_indices(self._c.natoms)] = d
    sorted_elements = [np.sort(_) for _ in s[atoms]]
    ret = []
    for i in range(len(atoms)):
        for j in range(i+1, len(atoms)):
            dist = np.linalg.norm(sorted_elements[i] - sorted_elements[j])
            ret.append([atoms[i], atoms[j], dist])
    ret = np.array(ret)
    return np.array(ret[:, 0], dtype=np.int), np.array(ret[:, 1], dtype=np.int), ret[:, 2]

%lprun -f get_site_similarity_CM_orig get_site_similarity_CM_orig(r, (5,7,5,7,5,7,5,7,5,7))
%timeit get_site_similarity_CM_orig(r, (5,7,5,7,5,7,5,7,5,7))

602 µs ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Timer unit: 1e-06 s

Total time: 0.010335 s
File: <ipython-input-206-04c9493dba82>
Function: get_site_similarity_CM_orig at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def get_site_similarity_CM_orig(self, nuclear_charges):
     3                                               """ Returns i, j, distance."""
     4         1         78.0     78.0      0.8      charges = self._c.nuclear_charges.copy().astype(np.float)
     5         1         17.0     17.0      0.2      charges[self._includeonly] = nuclear_charges
     6         1          4.0      4.0      0.0      atoms = self._includeonly
     7         1        303.0    303.0      2.9      a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
     8         1         26.0     26.0      0.3      s = np.zeros((self._c.natoms, self._c.natoms))
     9         1        231.0    231.0      2.2      s[n

In [97]:
# Use numba for loops, early exit
@numba.jit(nopython=True)
def numba_exit_norm(A, B, n, limit):
    delta = 0.
    for i in range(n):
        delta += (A[i] - B[i])**2
        if delta > limit:
            return limit + 1
    return delta**0.5

def get_site_similarity_CM(self, nuclear_charges):
    """ Returns i, j, distance."""
    charges = self._c.nuclear_charges.copy().astype(np.float)
    charges[self._includeonly] = nuclear_charges
    atoms = self._includeonly
    a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    s = np.zeros((self._c.natoms, self._c.natoms))
    s[np.tril_indices(self._c.natoms)] = a
    d = np.diag(s)
    s += s.T
    s[np.diag_indices(self._c.natoms)] = d
    sorted_elements = [np.sort(_) for _ in s[atoms]]
    limit = self._similarity_parameter**2.
    ret = []
    for i in range(len(atoms)):
        for j in range(i+1, len(atoms)):
            dist = numba_exit_norm(sorted_elements[i], sorted_elements[j], self._c.natoms, limit)
            ret.append([atoms[i], atoms[j], dist])
    ret = np.array(ret)
    return np.array(ret[:, 0], dtype=np.int), np.array(ret[:, 1], dtype=np.int), ret[:, 2]

In [77]:
numba_exit_norm(TESTVECS[0], TESTVECS[1], 18, 10)
%lprun -f get_site_similarity_CM get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))
%timeit get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))

250 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Timer unit: 1e-06 s

Total time: 0.002084 s
File: <ipython-input-69-b613c9ffd911>
Function: get_site_similarity_CM at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def get_site_similarity_CM(self, nuclear_charges):
    12                                               """ Returns i, j, distance."""
    13         1         38.0     38.0      1.8      charges = self._c.nuclear_charges.copy().astype(np.float)
    14         1         16.0     16.0      0.8      charges[self._includeonly] = nuclear_charges
    15         1          6.0      6.0      0.3      atoms = self._includeonly
    16         1        262.0    262.0     12.6      a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    17         1         17.0     17.0      0.8      s = np.zeros((self._c.natoms, self._c.natoms))
    18         1        109.0    109.0      5.2      s[np.tril_ind

In [155]:
# skip recurrent output
@numba.jit(nopython=True)
def numba_exit_norm(A, B, n, limit):
    delta = 0.
    for i in range(n):
        delta += (A[i] - B[i])**2
        if delta > limit:
            return limit + 1
    return delta**0.5

def get_site_similarity_CM(self, nuclear_charges):
    """ Returns i, j, distance."""
    charges = self._c.nuclear_charges.copy().astype(np.float)
    charges[self._includeonly] = nuclear_charges
    atoms = self._includeonly
    a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    s = np.zeros((self._c.natoms, self._c.natoms))
    s[np.tril_indices(self._c.natoms)] = a
    d = np.diag(s)
    s += s.T
    s[np.diag_indices(self._c.natoms)] = d
    sorted_elements = np.sort(s[atoms], axis=1)
    
    limit = self._similarity_parameter**2.
    npairs = int(self._nmodifiedatoms * (self._nmodifiedatoms-1)/2)
    ret = np.zeros(npairs)
    atomi, atomj = self._cache_site_similarity_indices
    
    for i in range(npairs):
        dist = numba_exit_norm(sorted_elements[atomi[i]], sorted_elements[atomj[i]], self._c.natoms, limit)
        ret[i] = dist
    return ret

In [156]:
numba_exit_norm(TESTVECS[0], TESTVECS[1], 18, 10)
%lprun -f get_site_similarity_CM get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))
%timeit get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))

401 µs ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Timer unit: 1e-06 s

Total time: 0.483879 s
File: <ipython-input-155-6baaf2341b43>
Function: get_site_similarity_CM at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def get_site_similarity_CM(self, nuclear_charges):
    12                                               """ Returns i, j, distance."""
    13         1         26.0     26.0      0.0      charges = self._c.nuclear_charges.copy().astype(np.float)
    14         1         19.0     19.0      0.0      charges[self._includeonly] = nuclear_charges
    15         1          4.0      4.0      0.0      atoms = self._includeonly
    16         1        108.0    108.0      0.0      a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    17         1         15.0     15.0      0.0      s = np.zeros((self._c.natoms, self._c.natoms))
    18         1        141.0    141.0      0.0      s[np.tril_in

In [167]:
# numpyify
@numba.jit(nopython=True)
def numba_exit_norm(A, B, n, limit):
    delta = 0.
    for i in range(n):
        delta += (A[i] - B[i])**2
        if delta > limit:
            return limit + 1
    return delta**0.5

def prepare_site_similarity_CM(self):
    self._sitesimCM_squareform_outcache = np.zeros((self._c.natoms, self._c.natoms))
    self._sitesimCM_squareform_mask = np.tril(np.ones((self._c.natoms, self._c.natoms),dtype=bool))

def get_site_similarity_CM(self, nuclear_charges):
    """ Returns i, j, distance."""
    charges = self._c.nuclear_charges.copy().astype(np.float)
    charges[self._includeonly] = nuclear_charges
    atoms = self._includeonly
    a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    
    # to squareform
    self._sitesimCM_squareform_outcache[self._sitesimCM_squareform_mask] = a
    self._sitesimCM_squareform_outcache.T[self._sitesimCM_squareform_mask] = a
    
    sorted_elements = np.sort(self._sitesimCM_squareform_outcache[atoms], axis=1)
    
    limit = self._similarity_parameter**2.
    npairs = int(self._nmodifiedatoms * (self._nmodifiedatoms-1)/2)
    ret = np.zeros(npairs)
    atomi, atomj = self._cache_site_similarity_indices
    
    for i in range(npairs):
        dist = numba_exit_norm(sorted_elements[atomi[i]], sorted_elements[atomj[i]], self._c.natoms, limit)
        ret[i] = dist
    return ret

In [173]:
numba_exit_norm(TESTVECS[0], TESTVECS[1], 18, 10)
prepare_site_similarity_CM(r)
%lprun -f get_site_similarity_CM get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))
%timeit get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))

286 µs ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Timer unit: 1e-06 s

Total time: 0.000884 s
File: <ipython-input-167-407f81e32edb>
Function: get_site_similarity_CM at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                           def get_site_similarity_CM(self, nuclear_charges):
    16                                               """ Returns i, j, distance."""
    17         1         23.0     23.0      2.6      charges = self._c.nuclear_charges.copy().astype(np.float)
    18         1         15.0     15.0      1.7      charges[self._includeonly] = nuclear_charges
    19         1          4.0      4.0      0.5      atoms = self._includeonly
    20         1        236.0    236.0     26.7      a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    21                                               
    22                                               # to squareform
    23         1         12.0     12.0    

In [209]:
# numpyify
@numba.jit(nopython=True)
def numba_exit_norm(A, B, n, limit):
    delta = 0.
    for i in range(n):
        delta += (A[i] - B[i])**2
        if delta > limit:
            return limit + 1
    return delta**0.5

@numba.jit(nopython=True)
def numba_loop(atomi, atomj, sorted_elements, natoms, limit):
    npairs = len(atomi)
    ret = np.zeros(npairs)
    for i in range(npairs):
        dist = numba_exit_norm(sorted_elements[atomi[i]], sorted_elements[atomj[i]], natoms, limit)
        ret[i] = dist
    return ret

def prepare_site_similarity_CM(self):
    self._sitesimCM_squareform_outcache = np.zeros((self._c.natoms, self._c.natoms))
    self._sitesimCM_squareform_mask = np.tril(np.ones((self._c.natoms, self._c.natoms),dtype=bool))
    self._sitesimCM_charges = self._c.nuclear_charges.copy().astype(np.float)

def get_site_similarity_CM(self, nuclear_charges):
    """ Returns i, j, distance."""
    charges = self._sitesimCM_charges
    charges[self._includeonly] = nuclear_charges
    a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    
    # to squareform
    self._sitesimCM_squareform_outcache[self._sitesimCM_squareform_mask] = a
    self._sitesimCM_squareform_outcache.T[self._sitesimCM_squareform_mask] = a
    
    sorted_elements = np.sort(self._sitesimCM_squareform_outcache[self._includeonly], axis=1, kind="stable")
    
    limit = self._similarity_parameter**2.
    atomi, atomj = self._cache_site_similarity_indices
    
    return numba_loop(atomi, atomj, sorted_elements, self._c.natoms, limit)

In [211]:
numba_exit_norm(TESTVECS[0], TESTVECS[1], 18, 10)
prepare_site_similarity_CM(r)
%lprun -f get_site_similarity_CM get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))
%timeit get_site_similarity_CM(r, (5,7,5,7,5,7,5,7,5,7))

42.1 µs ± 5.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Timer unit: 1e-06 s

Total time: 0.000442 s
File: <ipython-input-209-a2e3d9186af7>
Function: get_site_similarity_CM at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def get_site_similarity_CM(self, nuclear_charges):
    26                                               """ Returns i, j, distance."""
    27         1          6.0      6.0      1.4      charges = self._sitesimCM_charges
    28         1         18.0     18.0      4.1      charges[self._includeonly] = nuclear_charges
    29         1        317.0    317.0     71.7      a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
    30                                               
    31                                               # to squareform
    32         1         13.0     13.0      2.9      self._sitesimCM_squareform_outcache[self._sitesimCM_squareform_mask] = a
    33         1   